In [ ]:
%pip install pandas numpy seaborn matplotlib pycaret opencv-python

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.api as sm

from sklearn.preprocessing import LabelEncoder


import warnings 
warnings.filterwarnings(action='ignore')

In [ ]:
device_power_sensor_data = pd.read_csv('all_data/all_final_df/class_a_device_power_sensor.csv')
environmental_sensor_data = pd.read_csv('all_data/all_final_df/class_a_environmental_sensor.csv')

In [ ]:
device_power_sensor_data['time'] = pd.to_datetime(device_power_sensor_data['time'])
environmental_sensor_data['time'] = pd.to_datetime(environmental_sensor_data['time'])

In [ ]:
# 두 데이터셋 병합
total_data = pd.merge(
    environmental_sensor_data[['time', 'average_humidity(%)', 'average_co2(ppm)', 'average_illumination(lux)']],
    device_power_sensor_data[['time', 'socket_power(Wh)']],
    on='time',
    how='inner'
)

# 'time' 컬럼을 인덱스로 설정
total_data.set_index('time', inplace=True)

# 데이터 분할
split_date = pd.Timestamp('2024-05-02 00:00:00')
train_df = total_data[total_data.index < split_date]
test_df = total_data[total_data.index >= split_date]

print(total_data.info())


In [7]:
from pycaret.regression import *

# PyCaret 환경 설정
setup_config = setup(session_id=1,
            data=total_data,
            target='socket_power(Wh)',
            normalize = True)

# 모든 모델 비교
best_model = compare_models()

# 모델 최적화
tuned_model = tune_model(best_model)

# 모델 평가
evaluate_model(tuned_model)

KeyboardInterrupt: 

In [ ]:
# 모델 최종화
final_model = finalize_model(tuned_model)

# 예측
predictions = predict_model(final_model, fh=12)

In [ ]:
plot_model(final_model, plot='forecast')
plot_model(final_model, plot='diagnostic')

In [ ]:
save_model(final_model, 'final_time_series_model')